In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import http.client
from urllib.parse import urlencode, quote
import time
import datetime
from hashlib import sha1
from mpl_toolkits.basemap import Basemap
import random
import string
import os
import sys
import urllib as ul
import json
import numpy as np
import seaborn as sns
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from IPython.core.display import display, HTML, Image
Image(url='https://bcdn.se/images/resources/booli_logo.png')

ImportError: No module named 'mpl_toolkits.basemap'

In [5]:
# Load the dataframe
%store -r df
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8069 entries, 2210130 to 2013014
Data columns (total 29 columns):
additionalArea         8069 non-null float64
booliId                8069 non-null int64
constructionYear       7796 non-null float64
floor                  7462 non-null float64
isNewConstruction      8069 non-null float64
listPrice              8007 non-null float64
livingArea             8059 non-null float64
location               8069 non-null object
objectType             8069 non-null object
published              8069 non-null object
rent                   8041 non-null float64
rooms                  8064 non-null float64
soldDate               8069 non-null datetime64[ns]
soldPrice              8069 non-null int64
source                 8069 non-null object
url                    8069 non-null object
namedAreas             8069 non-null object
streetAddress          8069 non-null object
streetAddressNumber    8069 non-null object
streetAddressName      8069 non-nu

# 2. Visulization

In [2]:
#Set style
plt.style.use('fivethirtyeight')

plt.rc('grid', linestyle="--",alpha = 0.8)
plt.rc('font', size=14, family='sans-serif',style='normal',weight='normal')
plt.rc('legend', frameon=False,fontsize=14)
plt.rc('axes', labelsize=16, titlesize=20)
plt.rc('figure', figsize=(16,8),titlesize=20,facecolor ='white')
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)
plt.rc('lines', linewidth=3)
plt.rcParams['axes.formatter.limits'] = (-4,4)

NameError: name 'plt' is not defined

## 2.1 The Time Serie
How has the prices develop in time?

In [3]:
f, ax = plt.subplots(3,2,sharex = True)
agg = {'soldPrice':{'mean':'mean'},
       'soldPriceSqm':{'mean':'mean'},
       'listPriceSqm':{'mean':'mean'},
       'livingArea':{'mean':'mean'},
       'booliId':{'count':'count'},
       'rent':{'mean':'mean'}}
df.groupby('soldMonth').agg(agg).plot(ax=ax, kind='line',title='Average price and size',subplots=True)
plt.show()

NameError: name 'plt' is not defined

## 2.2 Construction Year distribution

In [4]:
binyears = 10
df['constructionYear'].plot(kind = 'hist',
                            bins = int((constructionYearRange[1]-constructionYearRange[0])/binyears),
                            xlim = constructionYearRange,
                            xticks=np.arange(constructionYearRange[0],constructionYearRange[1],binyears),
                            rot = 90,
                            title = 'construction year')
plt.show()

NameError: name 'df' is not defined

## 2.3. Area & Broker information
Which areas are the most expensive? Or which broker have the biggest difference between list price and sold price?

In [ ]:
# Start with city_district
f,ax = plt.subplots(1,2)
sns.boxplot(x = 'city_district',y='soldPriceSqm',data = df,ax = ax[0])
plt.xticks(rotation=90)
sns.boxplot(x = 'suburb',y='soldPriceSqm',data = df,ax = ax[1])
#ax.set_title('city_district boxplot')
plt.sca(ax[0])
plt.xticks(rotation=90)
plt.sca(ax[1])
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Reaggregate
namedAreas = df.groupby('namedAreas').agg(agg)
# Plot what we got, only areas with more than 10 objects
namedAreas = namedAreas[namedAreas.loc[:,('namedAreas','count')]>10].sort_values(by=('soldPriceSqm','mean'), ascending = False)
df_area = df[df['namedAreas'].isin(namedAreas.index.values)]
sns.boxplot(x = 'namedAreas',y='soldPriceSqm',data = df_area)
xt = plt.xticks(rotation=90)

In [ ]:
agg = {'broker':{'count':'count'},
       'soldPriceSqm':{'mean':'mean'},
       'soldPrice':{'mean':'mean'},
       'listPrice':{'mean':'mean'},
       'changedPrice':{'mean':'mean'},
       'changedPriceSqm':{'mean':'mean'}}
broker = df.groupby('broker').agg(agg)

In [ ]:
#Look only at larger brokers
broker = broker[broker.loc[:,('broker','count')]>10]
df_broker = df[df.loc[:,'broker'].isin(broker.index.values)]
broker = df_broker.groupby('broker').agg(agg)

In [ ]:
#Look only at larger brokers
broker = broker[broker.loc[:,('broker','count')]>10]
df_broker = df[df.loc[:,'broker'].isin(broker.index.values)]
broker = df_broker.groupby('broker').agg(agg)

In [ ]:
sns.boxplot(x = 'broker',y='soldPriceSqm',data = df_broker)
plt.xticks(rotation=90)
plt.title('Broker sqm-price')
plt.show()

In [ ]:
sns.boxplot(x = 'broker',y='soldPriceSqm',data = df_broker)
plt.xticks(rotation=90)
plt.title('Broker sqm-price')
plt.show()

The sold price per square meter had a dip in June, but the brokers listed price did not change from the month before leading to a big drop in the difference between the sold price and listed price (orange line above).

In [ ]:
#Check the broker distribution
f, ax = plt.subplots(1, 1)
(
    broker[('changedPrice','mean')].
    sort_values(ascending=False).
    plot(kind='bar',ax=ax,title='Average changedPrice')
)
plt.show()

In [ ]:
# 3. Correlation

# Look at distribution
df[['soldPrice','soldPriceSqm','rent','ocean']].hist(bins=100)
plt.show()

# log1p: log(h+1). This ensures that zero values are kept as zero values since log(0) is not defined. 
prices = pd.DataFrame({"price":df["soldPrice"], "log(price + 1)":np.log1p(df["soldPrice"])})
prices.hist(bins=100)
plt.show()

## 3.1 Numerical variables, correlation

### Take a look at the data

f, ax = plt.subplots(3, 2)
print(df.columns)
df.plot(kind='scatter',x='listPrice',y='soldPrice',ax = ax[0,0],title='listPrice')
df.plot(kind='scatter',x='livingArea',y='soldPrice',ax = ax[1,0],title='livingArea')
df.plot(kind='scatter',x='rent',y='soldPrice',ax = ax[2,0],title='rent')
df.plot(kind='scatter',x='soldPriceSqm',y='soldPrice',ax = ax[0,1],title='soldDate')
df.plot(kind='scatter',x='ocean',y='soldPrice',ax = ax[1,1],title='ocean')

plt.figure(1)
f, ax = plt.subplots(4, 2)
price = df['soldPrice'].values
ax[0, 0].scatter(df.listPrice.values, price)
ax[0, 0].set_title('listPrice')
ax[0, 1].scatter(df.livingArea.values, price)
ax[0, 1].set_title('livingArea')
ax[1, 0].scatter(df.ocean.values, price)
ax[1, 0].set_title('ocean')
ax[1, 1].scatter(df['constructionYear'].values, price)
ax[1, 1].set_title('constructionYear')
ax[2, 0].scatter(df.floor.values, price)
ax[2, 0].set_title('floor')
ax[2, 1].scatter(df.rent.values, price)
ax[2, 1].set_title('rent')
ax[3, 0].scatter(df.rooms.values, price)
ax[3, 0].set_title('rooms')
ax[3, 1].scatter(df.ocean.values, price)
ax[3, 1].set_title('ocean')
f.text(-0.01, 0.5, 'Sale Price', va='center', rotation='vertical', fontsize = 12)
plt.tight_layout()
plt.show()

### Correlation heatmap

corr = df.ix[:, ~df.columns.isin(['booliId','isNewConstruction'])].select_dtypes(include = ['float64', 'int64']).corr()
plt.figure(figsize=(8, 8))
sns.heatmap(corr, vmax=1, square=True)

print('Numerical variables Correlation\n')
print(corr['soldPriceSqm'].sort_values(ascending = False))
print('--------------------')
print(corr['soldPrice'].sort_values(ascending = False))

#Select features to check for skewness
from scipy.stats import skew
#numeric_feats = df.select_dtypes(include = ['float64', 'int64']).index
numeric_feats = df.dtypes[df.dtypes == "float64"].index
skewed_feats = df[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
print(skewed_feats)
dd = np.log1p(df[skewed_feats])
dd.hist()